# Instantiate `DietVision` class

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%reload_ext autoreload

from diet_vision import DietVision
dv_instance = DietVision()

ModuleNotFoundError: No module named 'cv2'

In [ ]:
dv_instance.upload_image('chocolate_cake.jpg')

In [4]:
dv_instance.find_mask_index_list([1170, 991, 1666, 1073])
# [0, 8, 32, 42, 45, 69, 72, 74]

[0, 8, 32, 42, 45, 69, 72, 74]

In [78]:
for idx, dvd in enumerate(dv_instance.diet_vision_dictionary):
    data = Image.fromarray(dvd['mask'])
    data.save(f'mask_1_fish_chips_{idx}.png')

In [21]:
import requests
import json

items = range(0, 42)
response = []
for item in items:
    url = f'https://api.nal.usda.gov/fdc/v1/foods/list?dataType=Foundation,SR%20Legacy&pageSize=200&pageNumber={item}&api_key=ljOu2YIUetK44MUIP1iW7PJzRGo7jdSTTJyzHA0k'
    res1 = requests.get(url)
    res2 = json.loads(res1.text) # list length 200
    
    for food in res2:
        response.append(food['description'])

food_dictionary = { 'foods': response }
len(food_dictionary['foods'])

# Serializing json
json_object = json.dumps(food_dictionary, indent=4)
 
# Writing to sample.json
with open('../static/js/usdaFoods.js', "w") as outfile:
    outfile.write(json_object)

In [55]:
combined_mask2 = dv_instance.diet_vision_dictionary[0]['mask'].copy()

for idx in range(1, len(dv_instance.diet_vision_dictionary)):
    curr_mask = dv_instance.diet_vision_dictionary[idx]['mask'].copy()
    combined_mask2 = np.logical_or(combined_mask2, curr_mask)

In [56]:
data = Image.fromarray(combined_mask2)
data.save('combined_mask2.png')

In [57]:
combined_mask3 = dv_instance.raw_diet_vision_dictionary[0]['mask'].copy()

for idx in range(1, len(dv_instance.raw_diet_vision_dictionary)):
    curr_mask = dv_instance.raw_diet_vision_dictionary[idx]['mask'].copy()
    combined_mask3 = np.logical_or(combined_mask3, curr_mask)

data = Image.fromarray(combined_mask3)
data.save('combined_mask3.png')

In [62]:
for idx in range(1, len(dv_instance.raw_diet_vision_dictionary)):
    nonzero_at = dv_instance.raw_diet_vision_dictionary[idx]['nonzero_at']

    if (1355, 967) in nonzero_at:
        print(f'idx = {idx}')

idx = 1
idx = 2


In [70]:
import numpy as np

raw_diet_vision_dictionary = [{
    'mask': mask['segmentation'],
    'area': mask['area'],
    'class': None,
    'attached_to': idx,
    'nonzero_at': set(np.rec.fromarrays(np.nonzero(mask['segmentation'])).tolist())
} for idx, mask in enumerate(sorted(dv_instance.sam_result, key=lambda x: x['area'], reverse=False))]

height, width = raw_diet_vision_dictionary[0]['mask'].shape

mask_dictionary = np.full((height, width), -1)

for idx, curr_dvd in enumerate(raw_diet_vision_dictionary):
    for pixel in curr_dvd['nonzero_at']:
        mask_dictionary[pixel] = idx

        # curr_dvd['attached_to'] = self.mask_dictionary[pixel]

In [72]:
import itertools as it
import numpy as np

masks = []
for idx in range(0, len(raw_diet_vision_dictionary)):
    mask = np.full((height, width), False)

    tups = np.where(mask_dictionary == idx)

    if tups[0].size > 0 and tups[0].size == tups[1].size:
        for row, col in it.zip_longest(tups[0], tups[1]):
            mask[row, col] = True

        masks.append(mask)

In [73]:
from PIL import Image

for idx in range(0, len(masks)):
    data = Image.fromarray(masks[idx])
    data.save(f'4_my_mask_{idx}.png')

combined_mask = masks[0]

for item in masks[1:]:
    curr_mask = item.copy()
    combined_mask = np.logical_or(combined_mask, curr_mask)

data = Image.fromarray(combined_mask)
data.save('4_combined_mask.png')

In [69]:
data = Image.fromarray(np.logical_or(masks[58], masks[61]))
data.save('test.png')

In [159]:
import glob
import os

if not os.path.exists('../static/images/overlays/'):
    os.mkdir('../static/images/overlays/')
else :
    for file in glob.glob('../static/images/overlays/**.txt'):
        os.remove(file)

In [162]:
aset = set([])
aset.add((1, 1))
aset.add((2, 2))
tuple(zip(*aset))

os.path.join('../app/', 'b', 'c')

'../app/b/c'